# Hello

In [1]:
from datasets import load_dataset
import sentencepiece as spm
import tensorflow as tf
import pickle
import os

In [3]:
# ============================================================================
# MOUNT GOOGLE DRIVE VÀ SETUP ĐƯỜNG DẪN
# ============================================================================

from google.colab import drive
import os
import pickle

# Mount Google Drive
drive.mount('/content/drive')

# Tạo thư mục lưu dataset trên Drive
DRIVE_PATH = '/content/drive/MyDrive/NLP_Dataset/WMT14_en_fr'
os.makedirs(DRIVE_PATH, exist_ok=True)

print(f"✅ Đã mount Google Drive")
print(f"📁 Thư mục lưu dataset: {DRIVE_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã mount Google Drive
📁 Thư mục lưu dataset: /content/drive/MyDrive/NLP_Dataset/WMT14_en_fr


# Import dataset từ hugging face

In [4]:
ds = load_dataset("wmt/wmt14", "de-en")

de-en/train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

de-en/train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

de-en/train-00002-of-00003.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

de-en/validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

de-en/test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [5]:
# Kiểm tra số lượng dữ liệu trong dataset
print(f"Số lượng dữ liệu trong tập train: {ds['train'].num_rows}")
print(f"Số lượng dữ liệu trong tập validation: {ds['validation'].num_rows}")
print(f"Số lượng dữ liệu trong tập test: {ds['test'].num_rows}")

Số lượng dữ liệu trong tập train: 4508785
Số lượng dữ liệu trong tập validation: 3000
Số lượng dữ liệu trong tập test: 3003


# Subset - Dataset full ~40M câu → không train nổi

# Train tokenizer (SentencePiece, joint vocab)

In [ ]:
# Định nghĩa đường dẫn cho SentencePiece model
SPM_MODEL_PATH = os.path.join(DRIVE_PATH, "spm.model")

def extract_text_from_dataset(dataset, lang, max_lines=200_000):
    """Trích xuất text từ HuggingFace dataset"""
    texts = []
    for i, item in enumerate(dataset):
        if i >= max_lines:
            break
        texts.append(item['translation'][lang])
    return texts

# Kiểm tra xem tokenizer đã được train chưa
if os.path.exists(SPM_MODEL_PATH):
    print("📁 Tìm thấy SentencePiece model trên Drive!")
    print(f"   Loading từ: {SPM_MODEL_PATH}")
else:
    print("⚠️ Chưa có SentencePiece model. Đang train...")

    # Trích xuất text để train tokenizer từ HuggingFace dataset
    de_text = extract_text_from_dataset(ds['train'], "de")
    en_text = extract_text_from_dataset(ds['train'], "en")
    print(f"   Đã trích xuất {len(de_text):,} câu DE và {len(en_text):,} câu EN")

    # Tạo file input cho SentencePiece
    spm_input_path = os.path.join(DRIVE_PATH, "spm_input.txt")
    with open(spm_input_path, "w", encoding="utf-8") as f:
        for line in de_text + en_text:
            f.write(line + "\n")

    # Train SentencePiece
    spm.SentencePieceTrainer.train(
        input=spm_input_path,
        model_prefix=os.path.join(DRIVE_PATH, "spm"),
        vocab_size=16000,
        model_type="bpe",
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3
    )
    print(f"✅ Đã train và lưu SentencePiece model lên Drive")

# Load SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load(SPM_MODEL_PATH)
print(f"✅ Đã load SentencePiece model (vocab size: {sp.get_piece_size()})")

# Encode câu (RẤT QUAN TRỌNG)

In [ ]:
def encode_pair(src_text, tgt_text):
    """
    Encode một cặp câu source-target

    Args:
        src_text: Câu nguồn (German)
        tgt_text: Câu đích (English)

    Returns:
        src_ids: Token IDs của câu nguồn
        tgt_in: Decoder input (<bos> + target)
        tgt_out: Decoder output (target + <eos>)
    """
    src_ids = sp.encode(src_text)
    tgt_ids = sp.encode(tgt_text)

    # Decoder input và output
    tgt_in  = [2] + tgt_ids        # <bos> + y
    tgt_out = tgt_ids + [3]        # y + <eos>

    return src_ids, tgt_in, tgt_out

# Test thử
sample = ds['train'][0]
src, tgt_in, tgt_out = encode_pair(sample['translation']['de'], sample['translation']['en'])
print(f"Sample DE: {sample['translation']['de'][:50]}...")
print(f"Sample EN: {sample['translation']['en'][:50]}...")
print(f"Encoded src: {src[:10]}... (len={len(src)})")
print(f"Encoded tgt_in: {tgt_in[:10]}... (len={len(tgt_in)})")


# Tạo Dataset Cho Training

In [ ]:
# ============================================================================
# TẠO TF.DATA.DATASET TỪ HUGGINGFACE DATASET
# ============================================================================

BATCH_SIZE = 64
MAX_LEN = 100

def create_tf_dataset(hf_dataset, batch_size=64, max_len=100, shuffle=False):
    """
    Tạo tf.data.Dataset từ HuggingFace dataset

    Args:
        hf_dataset: HuggingFace dataset split (ví dụ: ds['train'])
        batch_size: Kích thước batch
        max_len: Độ dài tối đa của câu
        shuffle: Có shuffle data không

    Returns:
        tf.data.Dataset với format (src, tgt_in, tgt_out)
    """

    def generator():
        for item in hf_dataset:
            src_ids, tgt_in, tgt_out = encode_pair(
                item['translation']['de'], 
                item['translation']['en']
            )

            # Filter theo độ dài
            if len(src_ids) <= max_len and len(tgt_in) <= max_len:
                yield (src_ids, tgt_in, tgt_out)

    # Tạo dataset từ generator
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None,), dtype=tf.int32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)
        )
    )

    # Shuffle nếu cần
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10000)

    # Padded batch
    dataset = dataset.padded_batch(
        batch_size,
        padded_shapes=([None], [None], [None]),
        padding_values=(0, 0, 0)  # PAD_ID = 0
    )

    # Prefetch để tăng tốc
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Tạo train và valid dataset
print("⏳ Đang tạo tf.data.Dataset...")
train_ds = create_tf_dataset(ds['train'], batch_size=BATCH_SIZE, max_len=MAX_LEN, shuffle=True)
valid_ds = create_tf_dataset(ds['validation'], batch_size=BATCH_SIZE, max_len=MAX_LEN, shuffle=False)

# Test một batch
for src, tgt_in, tgt_out in train_ds.take(1):
    print(f"\n✅ Dataset đã sẵn sàng!")
    print(f"   Batch shape - src: {src.shape}, tgt_in: {tgt_in.shape}, tgt_out: {tgt_out.shape}")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Max sequence length: {MAX_LEN}")


# Xây dựng Model LSTM Encoder-Decoder

## Encoder
Encoder xử lý câu nguồn (French) và tạo ra Context Vector.

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    """
    LSTM Encoder: Xử lý câu nguồn và tạo Context Vector

    Args:
        input_dim: Kích thước vocabulary nguồn
        emb_dim: Kích thước embedding
        hid_dim: Kích thước hidden state
        n_layers: Số layers LSTM
        dropout: Tỷ lệ dropout
    """
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Embedding layer: Chuyển token index thành vector
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=0)

        # LSTM layer
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):
        """
        Args:
            src: [batch_size, src_len] - Token indices
            src_len: [batch_size] - Độ dài thực của mỗi câu

        Returns:
            hidden: [n_layers, batch_size, hid_dim]
            cell: [n_layers, batch_size, hid_dim]
        """
        # Embedding
        embedded = self.dropout(self.embedding(src))
        # embedded: [batch_size, src_len, emb_dim]

        # Pack để bỏ qua padding
        packed_embedded = pack_padded_sequence(
            embedded,
            src_len.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        # Đưa qua LSTM
        packed_outputs, (hidden, cell) = self.rnn(packed_embedded)

        # Hidden và Cell state sẽ được dùng làm context vector cho Decoder
        return hidden, cell

print("✅ Encoder đã sẵn sàng!")

## Decoder
Decoder nhận Context Vector từ Encoder và sinh ra câu đích (English) từng token một.

In [ ]:
class Decoder(nn.Module):
    """
    LSTM Decoder: Sinh câu đích từ Context Vector

    Args:
        output_dim: Kích thước vocabulary đích
        emb_dim: Kích thước embedding
        hid_dim: Kích thước hidden state
        n_layers: Số layers LSTM
        dropout: Tỷ lệ dropout
    """
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Embedding cho ngôn ngữ đích (English)
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=0)

        # LSTM layer
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)

        # Output layer: map từ hidden state ra vocabulary size
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        """
        Forward pass cho 1 bước decode

        Args:
            input: [batch_size] - Token index tại bước hiện tại
            hidden: [n_layers, batch_size, hid_dim] - Hidden state từ bước trước
            cell: [n_layers, batch_size, hid_dim] - Cell state từ bước trước

        Returns:
            prediction: [batch_size, output_dim] - Xác suất cho mỗi token
            hidden: [n_layers, batch_size, hid_dim]
            cell: [n_layers, batch_size, hid_dim]
        """
        # input: [batch_size] -> [batch_size, 1]
        input = input.unsqueeze(1)

        # Embedding
        embedded = self.dropout(self.embedding(input))
        # embedded: [batch_size, 1, emb_dim]

        # LSTM
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [batch_size, 1, hid_dim]

        # Fully connected
        prediction = self.fc_out(output.squeeze(1))
        # prediction: [batch_size, output_dim]

        return prediction, hidden, cell

print("✅ Decoder đã sẵn sàng!")

## Seq2Seq Model
Kết hợp Encoder và Decoder với cơ chế Teacher Forcing.

In [ ]:
import random

class Seq2Seq(nn.Module):
    """
    Sequence-to-Sequence Model kết hợp Encoder và Decoder

    Args:
        encoder: Encoder module
        decoder: Decoder module
    """
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
        """
        Forward pass cho toàn bộ sequence

        Args:
            src: [batch_size, src_len] - Câu nguồn (đã padding)
            src_len: [batch_size] - Độ dài thực của câu nguồn
            trg: [batch_size, trg_len] - Câu đích (đã padding)
            teacher_forcing_ratio: Xác suất sử dụng ground truth

        Returns:
            outputs: [batch_size, trg_len, output_dim] - Predictions
        """
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor chứa kết quả dự đoán
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).cuda()

        # Encoder
        hidden, cell = self.encoder(src, src_len)

        # Token đầu tiên cho decoder là <bos>
        input = trg[:, 0]

        # Decode từng bước
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output

            # Teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs

print("✅ Seq2Seq Model đã sẵn sàng!")

# Callbacks - Điều khiển quá trình huấn luyện

- **ReduceLROnPlateau:** Giảm learning rate khi validation loss không cải thiện
- **EarlyStopping:** Dừng sớm để tránh overfitting
- **ASK:** Cho phép người dùng kiểm soát quá trình training

In [ ]:
import time

class ASK:
    """
    Custom callback để hỏi user có muốn tiếp tục training không
    """
    def __init__(self, epochs, ask_epoch):
        self.ask_epoch = ask_epoch
        self.epochs = epochs
        self.ask = True
        self.stop_training = False
        self.start_time = None

    def on_train_begin(self):
        if self.ask_epoch == 0:
            print('ask_epoch = 0, sẽ được set thành 1')
            self.ask_epoch = 1
        if self.ask_epoch >= self.epochs:
            print(f'ask_epoch >= epochs, sẽ train hết {self.epochs} epochs')
            self.ask = False
        if self.epochs == 1:
            self.ask = False
        else:
            print(f'Training sẽ chạy đến epoch {self.ask_epoch}, sau đó sẽ hỏi tiếp tục')
        self.start_time = time.time()

    def on_train_end(self):
        tr_duration = time.time() - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        print(f'⏱️ Tổng thời gian training: {int(hours)}h {int(minutes)}m {seconds:.2f}s')

    def on_epoch_end(self, epoch):
        if self.ask and epoch + 1 == self.ask_epoch:
            print('\n Nhập H để dừng, hoặc số epochs muốn train thêm:')
            ans = input()
            if ans.lower() == 'h' or ans == '0':
                print(f'Training dừng tại epoch {epoch+1}')
                self.stop_training = True
                return True
            else:
                self.ask_epoch += int(ans)
                print(f'Sẽ tiếp tục đến epoch {self.ask_epoch}')
        return False


class ReduceLROnPlateau:
    """Giảm learning rate khi val_loss không cải thiện"""
    def __init__(self, optimizer, factor=0.5, patience=2, verbose=True, min_lr=1e-7):
        self.optimizer = optimizer
        self.factor = factor
        self.patience = patience
        self.verbose = verbose
        self.min_lr = min_lr
        self.best_loss = float('inf')
        self.counter = 0

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                for param_group in self.optimizer.param_groups:
                    old_lr = param_group['lr']
                    new_lr = max(old_lr * self.factor, self.min_lr)
                    param_group['lr'] = new_lr
                    if self.verbose and new_lr != old_lr:
                        print(f'\n📉 ReduceLR: {old_lr:.6f} → {new_lr:.6f}')
                self.counter = 0


class EarlyStopping:
    """Dừng training sớm khi val_loss không cải thiện"""
    def __init__(self, patience=4, verbose=True, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def step(self, val_loss, model):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            if self.verbose:
                print(f'✅ val_loss improved: {val_loss:.4f}')
        else:
            self.counter += 1
            if self.verbose:
                print(f'⚠️  val_loss không cải thiện ({self.counter}/{self.patience})')
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f'\n⛔ Early stopping! Đang restore best model...')
                if self.best_model_state is not None:
                    model.load_state_dict(self.best_model_state)
        return self.early_stop


def create_callbacks(optimizer, epochs, ask_epoch):
    """Tạo dict chứa các callbacks"""
    return {
        'reduce_lr': ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True),
        'early_stop': EarlyStopping(patience=4, verbose=True),
        'ask': ASK(epochs, ask_epoch)
    }

print("✅ Callbacks đã sẵn sàng!")

# Khởi tạo Model và Training Components

Khởi tạo Encoder, Decoder, Seq2Seq, Optimizer, Loss và Callbacks.

In [ ]:
# ============================================================================
# KHỞI TẠO MODEL
# ============================================================================

# Hyperparameters
VOCAB_SIZE = sp.get_piece_size()  # Từ SentencePiece
EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DROPOUT = 0.5
PAD_IDX = 0  # padding index trong SentencePiece
BOS_IDX = 2  # <bos> index
EOS_IDX = 3  # <eos> index

# Encoder (French → Context Vector)
encoder = Encoder(
    input_dim=VOCAB_SIZE,
    emb_dim=EMB_DIM,
    hid_dim=HID_DIM,
    n_layers=N_LAYERS,
    dropout=DROPOUT
)

# Decoder (Context Vector → English)
decoder = Decoder(
    output_dim=VOCAB_SIZE,
    emb_dim=EMB_DIM,
    hid_dim=HID_DIM,
    n_layers=N_LAYERS,
    dropout=DROPOUT
)

# Seq2Seq Model
model = Seq2Seq(encoder, decoder).cuda()

# Optimizer với weight decay (L2 regularization)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=1e-5
)

# Loss function với label smoothing
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX, label_smoothing=0.1)

# Callbacks
N_EPOCHS = 20
callbacks = create_callbacks(optimizer, epochs=N_EPOCHS, ask_epoch=5)

# In thông tin model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ Model khởi tạo thành công!")
print(f"📊 Số parameters: {count_parameters(model):,}")
print(f"📊 Vocab size: {VOCAB_SIZE}")
print(f"📊 Embedding dim: {EMB_DIM}")
print(f"📊 Hidden dim: {HID_DIM}")
print(f"📊 Num layers: {N_LAYERS}")
print(f"📊 Dropout: {DROPOUT}")

# Hàm Train và Evaluate

In [ ]:
def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio=0.5):
    """
    Huấn luyện model cho 1 epoch

    Args:
        model: Seq2Seq model
        iterator: tf.data.Dataset
        optimizer: Adam optimizer
        criterion: Loss function
        clip: Gradient clipping value
        teacher_forcing_ratio: Tỷ lệ teacher forcing

    Returns:
        epoch_loss: Loss trung bình
    """
    model.train()
    epoch_loss = 0
    num_batches = 0

    for batch in iterator:
        src, tgt_in, tgt_out = batch

        # Chuyển sang PyTorch tensor
        src = torch.LongTensor(src.numpy()).cuda()
        tgt_in = torch.LongTensor(tgt_in.numpy()).cuda()
        tgt_out = torch.LongTensor(tgt_out.numpy()).cuda()

        # Tính độ dài thực của src (không tính padding)
        src_len = (src != PAD_IDX).sum(dim=1)

        optimizer.zero_grad()

        # Forward pass
        output = model(src, src_len, tgt_in, teacher_forcing_ratio)
        # output: [batch_size, tgt_len, vocab_size]

        # Reshape để tính loss
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Bỏ token đầu
        tgt_out = tgt_out[:, 1:].reshape(-1)

        # Loss
        loss = criterion(output, tgt_out)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        num_batches += 1

    return epoch_loss / num_batches


def evaluate(model, iterator, criterion):
    """
    Đánh giá model trên validation set

    Args:
        model: Seq2Seq model
        iterator: tf.data.Dataset
        criterion: Loss function

    Returns:
        epoch_loss: Loss trung bình
    """
    model.eval()
    epoch_loss = 0
    num_batches = 0

    with torch.no_grad():
        for batch in iterator:
            src, tgt_in, tgt_out = batch

            src = torch.LongTensor(src.numpy()).cuda()
            tgt_in = torch.LongTensor(tgt_in.numpy()).cuda()
            tgt_out = torch.LongTensor(tgt_out.numpy()).cuda()

            src_len = (src != PAD_IDX).sum(dim=1)

            # Forward với teacher_forcing_ratio = 0
            output = model(src, src_len, tgt_in, teacher_forcing_ratio=0)

            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt_out = tgt_out[:, 1:].reshape(-1)

            loss = criterion(output, tgt_out)

            epoch_loss += loss.item()
            num_batches += 1

    return epoch_loss / num_batches


def epoch_time(start_time, end_time):
    """Tính thời gian epoch"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

print("✅ Các hàm Train/Evaluate đã sẵn sàng!")

# Training Loop

Huấn luyện model với các kỹ thuật chống overfitting:
- Teacher Forcing Decay
- Label Smoothing
- Early Stopping
- ReduceLROnPlateau

In [ ]:
import math

# ============================================================================
# TRAINING LOOP
# ============================================================================

CLIP = 1  # Gradient clipping

# Teacher Forcing Decay
TEACHER_FORCING_START = 0.5
TEACHER_FORCING_DECAY = 0.95
TEACHER_FORCING_MIN = 0.2

# Lưu lịch sử training
history = {
    'train_loss': [],
    'val_loss': [],
    'train_ppl': [],
    'val_ppl': [],
    'teacher_forcing': []
}

print(f"🚀 Bắt đầu huấn luyện!")
print(f"   - Epochs: {N_EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print("="*70)

# Bắt đầu training
callbacks['ask'].on_train_begin()
teacher_forcing_ratio = TEACHER_FORCING_START

for epoch in range(N_EPOCHS):
    start_time = time.time()

    # Training
    train_loss = train(
        model,
        train_ds,
        optimizer,
        criterion,
        CLIP,
        teacher_forcing_ratio=teacher_forcing_ratio
    )

    # Validation
    valid_loss = evaluate(model, valid_ds, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Tính Perplexity
    train_ppl = math.exp(train_loss)
    val_ppl = math.exp(valid_loss)

    # Lưu vào history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(valid_loss)
    history['train_ppl'].append(train_ppl)
    history['val_ppl'].append(val_ppl)
    history['teacher_forcing'].append(teacher_forcing_ratio)

    # In kết quả
    print(f'\nEpoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s | TF: {teacher_forcing_ratio:.3f}')
    print(f'   Train Loss: {train_loss:.3f} | Train PPL: {train_ppl:7.3f}')
    print(f'   Val. Loss:  {valid_loss:.3f} | Val. PPL:  {val_ppl:7.3f}')

    # Callbacks
    callbacks['reduce_lr'].step(valid_loss)

    if callbacks['early_stop'].step(valid_loss, model):
        print(f'\n⛔ Early stopping sau {epoch+1} epochs!')
        break

    if callbacks['ask'].on_epoch_end(epoch):
        break

    # Giảm teacher forcing
    teacher_forcing_ratio = max(TEACHER_FORCING_MIN, teacher_forcing_ratio * TEACHER_FORCING_DECAY)
    print("-"*70)

callbacks['ask'].on_train_end()

print("="*70)
print(f'🎉 Hoàn thành huấn luyện!')
print(f'🎯 Best validation loss: {callbacks["early_stop"].best_loss:.3f}')

In [ ]:
# ============================================================================
# LƯU MODEL VÀ HÀM DỊCH
# ============================================================================

# Lưu model lên Drive
MODEL_SAVE_PATH = os.path.join(DRIVE_PATH, "best_model.pt")
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"💾 Model đã lưu tại: {MODEL_SAVE_PATH}")

def translate(sentence, model, sp, max_len=50):
    """
    Dịch câu từ French sang English

    Args:
        sentence: Câu tiếng Pháp
        model: Trained Seq2Seq model
        sp: SentencePiece processor
        max_len: Độ dài tối đa output

    Returns:
        translated: Câu tiếng Anh
    """
    model.eval()

    # Encode source sentence
    src_ids = sp.encode(sentence)
    src_tensor = torch.LongTensor([src_ids]).cuda()
    src_len = torch.LongTensor([len(src_ids)]).cuda()

    with torch.no_grad():
        # Encoder
        hidden, cell = model.encoder(src_tensor, src_len)

        # Decoder - bắt đầu với <bos>
        trg_indexes = [BOS_IDX]

        for _ in range(max_len):
            trg_tensor = torch.LongTensor([trg_indexes[-1]]).cuda()

            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            if pred_token == EOS_IDX:
                break

    # Decode về text (bỏ <bos> và <eos>)
    trg_tokens = trg_indexes[1:-1] if trg_indexes[-1] == EOS_IDX else trg_indexes[1:]
    translated = sp.decode(trg_tokens)

    return translated

print("✅ Hàm translate đã sẵn sàng!")

In [ ]:
# ============================================================================
# TEST TRANSLATION
# ============================================================================

test_sentences = [
    "Eine Gruppe von Menschen steht vor einem Iglu.",
    "Ein Mann mit einem orangefarbenen Hut schaut auf etwas.",
    "Eine Frau geht mit ihrem Hund spazieren.",
    "Ein Kind in einem rosa Kleid geht die Treppe hinauf.",
    "Der Hund spielt mit einem Ball im Garten."
]

print("🔍 Test Translation (German → English):\n")
print("="*70)

for i, sentence in enumerate(test_sentences, 1):
    translation = translate(sentence, model, sp)
    print(f"\n📝 Câu {i}:")
    print(f"   DE: {sentence}")
    print(f"   EN: {translation}")
    print("-"*70)

print("\n✅ Hoàn thành!")

# Đánh giá BLEU Score

BLEU (Bilingual Evaluation Understudy) là metric tiêu chuẩn để đánh giá chất lượng dịch máy.

| BLEU Score | Đánh giá |
|------------|----------|
| < 10 | ❌ Rất tệ |
| 10 - 20 | ⚠️ Tệ |
| 20 - 30 | ⚠️ Chấp nhận |
| 30 - 40 | ✅ Tốt |
| 40 - 50 | 🌟 Rất tốt |
| > 50 | 🏆 Xuất sắc |

In [ ]:
import nltk
nltk.download('punkt')

from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
import numpy as np

def calculate_bleu(references, hypothesis, weights=(0.25, 0.25, 0.25, 0.25)):
    """Tính BLEU score cho 1 câu"""
    smoothing = SmoothingFunction().method1
    return sentence_bleu(
        references,
        hypothesis,
        weights=weights,
        smoothing_function=smoothing
    )


def evaluate_bleu_on_dataset(model, hf_dataset, sp, max_samples=None):
    """
    Tính BLEU score trên dataset

    Args:
        model: Trained Seq2Seq model
        hf_dataset: HuggingFace dataset split
        sp: SentencePiece processor
        max_samples: Số mẫu tối đa (None = tất cả)

    Returns:
        results: Dict chứa BLEU scores
        all_references: List references
        all_hypotheses: List hypotheses
    """
    model.eval()

    all_references = []
    all_hypotheses = []

    bleu_1_scores = []
    bleu_2_scores = []
    bleu_3_scores = []
    bleu_4_scores = []

    print("🔄 Đang tính BLEU score...")

    samples = hf_dataset.select(range(min(max_samples, len(hf_dataset)))) if max_samples else hf_dataset

    for i, item in enumerate(samples):
        # Reference (ground truth English)
        ref_tokens = sp.encode(item['translation']['en'], out_type=str)
        all_references.append([ref_tokens])

        # Hypothesis (predicted English)
        translation = translate(item['translation']['de'], model, sp)
        hyp_tokens = sp.encode(translation, out_type=str)
        all_hypotheses.append(hyp_tokens)

        # BLEU scores
        bleu_1 = calculate_bleu([ref_tokens], hyp_tokens, weights=(1, 0, 0, 0))
        bleu_2 = calculate_bleu([ref_tokens], hyp_tokens, weights=(0.5, 0.5, 0, 0))
        bleu_3 = calculate_bleu([ref_tokens], hyp_tokens, weights=(1/3, 1/3, 1/3, 0))
        bleu_4 = calculate_bleu([ref_tokens], hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25))

        bleu_1_scores.append(bleu_1)
        bleu_2_scores.append(bleu_2)
        bleu_3_scores.append(bleu_3)
        bleu_4_scores.append(bleu_4)

        if (i + 1) % 500 == 0:
            print(f"   Đã xử lý {i+1} câu...")

    # Corpus BLEU
    smoothing = SmoothingFunction().method1
    corpus_bleu_score = corpus_bleu(
        all_references,
        all_hypotheses,
        smoothing_function=smoothing
    )

    results = {
        'BLEU-1': np.mean(bleu_1_scores) * 100,
        'BLEU-2': np.mean(bleu_2_scores) * 100,
        'BLEU-3': np.mean(bleu_3_scores) * 100,
        'BLEU-4': np.mean(bleu_4_scores) * 100,
        'Corpus BLEU': corpus_bleu_score * 100
    }

    print(f"✅ Hoàn thành tính BLEU trên {len(samples)} câu!")

    return results, all_references, all_hypotheses

print("✅ BLEU evaluation functions đã sẵn sàng!")

In [ ]:
# ============================================================================
# TÍNH BLEU SCORE TRÊN VALIDATION SET
# ============================================================================

print("="*70)
print("📊 ĐÁNH GIÁ BLEU SCORE TRÊN VALIDATION SET")
print("="*70)

# Tính BLEU (giới hạn 1000 mẫu để nhanh hơn)
bleu_results, references, hypotheses = evaluate_bleu_on_dataset(
    model=model,
    hf_dataset=ds['validation'],
    sp=sp,
    max_samples=1000
)

# In kết quả
print(f"\n{'='*70}")
print(f"🎯 BLEU SCORES TRÊN VALIDATION SET")
print(f"{'='*70}")
for metric, score in bleu_results.items():
    print(f"   {metric:15s}: {score:6.2f}%")

# Đánh giá chất lượng
bleu_4 = bleu_results['BLEU-4']
if bleu_4 < 10:
    quality = "❌ Rất tệ - Không sử dụng được"
elif bleu_4 < 20:
    quality = "⚠️  Tệ - Hiểu được 1 phần"
elif bleu_4 < 30:
    quality = "⚠️  Chấp nhận được - Hiểu được ý chính"
elif bleu_4 < 40:
    quality = "✅ Tốt - Dịch khá tốt"
elif bleu_4 < 50:
    quality = "🌟 Rất tốt - Gần với human translation"
else:
    quality = "🏆 Xuất sắc - Ngang human translator"

print(f"\n   BLEU-4: {bleu_4:.2f}% → {quality}")
print(f"{'='*70}")

In [ ]:
# ============================================================================
# VẼ BIỂU ĐỒ KẾT QUẢ
# ============================================================================

import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Kết quả huấn luyện LSTM Encoder-Decoder (DE → EN)', fontsize=14, fontweight='bold')

# (a) Loss
ax1 = axes[0, 0]
ax1.plot(range(1, len(history['train_loss'])+1), history['train_loss'],
         label='Train Loss', marker='o', linewidth=2, color='#2196F3')
ax1.plot(range(1, len(history['val_loss'])+1), history['val_loss'],
         label='Val Loss', marker='s', linewidth=2, color='#F44336')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('(a) Training & Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# (b) Perplexity
ax2 = axes[0, 1]
ax2.plot(range(1, len(history['train_ppl'])+1), history['train_ppl'],
         label='Train PPL', marker='o', linewidth=2, color='#2196F3')
ax2.plot(range(1, len(history['val_ppl'])+1), history['val_ppl'],
         label='Val PPL', marker='s', linewidth=2, color='#F44336')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Perplexity')
ax2.set_title('(b) Training & Validation Perplexity')
ax2.legend()
ax2.grid(True, alpha=0.3)

# (c) BLEU Scores
ax3 = axes[1, 0]
bleu_types = ['BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4']
bleu_values = [bleu_results[key] for key in bleu_types]
colors = ['#3498DB', '#27AE60', '#F39C12', '#E74C3C']
bars = ax3.bar(bleu_types, bleu_values, color=colors, alpha=0.85, edgecolor='black')
ax3.set_ylabel('BLEU Score (%)')
ax3.set_title('(c) BLEU-n Scores')
ax3.set_ylim([0, max(bleu_values) * 1.3])
for bar, value in zip(bars, bleu_values):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
             f'{value:.1f}%', ha='center', fontsize=10, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# (d) Teacher Forcing Decay
ax4 = axes[1, 1]
ax4.plot(range(1, len(history['teacher_forcing'])+1), history['teacher_forcing'],
         marker='v', linewidth=2, color='#FF9800')
ax4.fill_between(range(1, len(history['teacher_forcing'])+1), 0, history['teacher_forcing'],
                 alpha=0.2, color='#FF9800')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Teacher Forcing Ratio')
ax4.set_title('(d) Teacher Forcing Decay')
ax4.set_ylim([0, 0.6])
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(DRIVE_PATH, 'training_results.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Biểu đồ đã được lưu tại: {DRIVE_PATH}/training_results.png")